# Actividad 6

Joel Fayam Rivera Peña

2076323  Grupo: 42

Estadistica Aplicada

## Librerias y datos

In [ ]:
#Librerias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from lifelines import CoxPHFitter

In [ ]:
datos=pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/desercion_laboral.csv')

In [ ]:
datos

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,departamento,salario
0,3.8,5.3,2,157,3,0,1,0,ventas,bajo
1,8.0,8.6,5,262,6,0,1,0,ventas,medio
2,1.1,8.8,7,272,4,0,1,0,ventas,medio
3,7.2,8.7,5,223,5,0,1,0,ventas,bajo
4,3.7,5.2,2,159,3,0,1,0,ventas,bajo
...,...,...,...,...,...,...,...,...,...,...
14994,4.0,5.7,2,151,3,0,1,0,soporte,bajo
14995,3.7,4.8,2,160,3,0,1,0,soporte,bajo
14996,3.7,5.3,2,143,3,0,1,0,soporte,bajo
14997,1.1,9.6,6,280,4,0,1,0,soporte,bajo


## Depurar la base de datos y estadística descriptiva

In [ ]:
datos.isnull().sum()

satisfaccion           0
evaluacion             0
proyectos              0
promedio_horas_mes     0
semestre               0
accidente              0
renunciar              0
ascenso_5years         0
departamento          26
salario                0
dtype: int64

In [ ]:
#Eliminamos los datos vacios debido a que al contar con 15,000 elementos, perder 26 no afectara al estudio de la muestra.
datos=pd.DataFrame(datos.dropna())

In [ ]:
datos.isnull().sum()

satisfaccion          0
evaluacion            0
proyectos             0
promedio_horas_mes    0
semestre              0
accidente             0
renunciar             0
ascenso_5years        0
departamento          0
salario               0
dtype: int64

In [ ]:
datos.duplicated().sum()

2982

In [ ]:
# Eliminamos los datos duplicados
datos=datos.drop_duplicates(keep='first').reset_index(drop=True)


In [ ]:
datos.duplicated().sum()

0

In [ ]:
datos.dtypes

satisfaccion          float64
evaluacion            float64
proyectos               int64
promedio_horas_mes      int64
semestre                int64
accidente               int64
renunciar               int64
ascenso_5years          int64
departamento           object
salario                object
dtype: object

In [ ]:
dummies_departamento=pd.get_dummies(datos['departamento']).astype(int)

In [ ]:
dummies_departamento

,Investigación,administración,contabilidad,desarrollo de producto,informática,mantenimiento,mercadotecnia,recursos humanos,soporte,ventas
0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...
11986,0,0,0,1,0,0,0,0,0,0
11987,0,0,0,1,0,0,0,0,0,0
11988,0,0,0,1,0,0,0,0,0,0
11989,0,0,0,1,0,0,0,0,0,0


In [ ]:
dummies_salario=pd.get_dummies(datos['salario']).astype(int)

In [ ]:
dummies_salario

,alto,bajo,medio
0,0,1,0
1,0,0,1
2,0,0,1
3,0,1,0
4,0,1,0
...,...,...,...
11986,0,1,0
11987,0,1,0
11988,0,1,0
11989,0,1,0


In [ ]:
datos=pd.concat([datos,dummies_departamento,dummies_salario],axis=1)

In [ ]:
#Eliminamos las columnas de departamento y salario ya que las volvimos numéricas y además eliminamos el departamento de ventas y el salario bajo para tener n-1 dummies
datos.drop(['departamento','salario','ventas','bajo'],axis=1,inplace=True)

In [ ]:
datos.head()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,Investigación,administración,contabilidad,desarrollo de producto,informática,mantenimiento,mercadotecnia,recursos humanos,soporte,alto,medio
0,3.8,5.3,2,157,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,8.0,8.6,5,262,6,0,1,0,0,0,0,0,0,0,0,0,0,0,1
2,1.1,8.8,7,272,4,0,1,0,0,0,0,0,0,0,0,0,0,0,1
3,7.2,8.7,5,223,5,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,3.7,5.2,2,159,3,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Estadísticas descriptivas
datos.describe()

,satisfaccion,evaluacion,proyectos,promedio_horas_mes,semestre,accidente,renunciar,ascenso_5years,Investigación,administración,contabilidad,desarrollo de producto,informática,mantenimiento,mercadotecnia,recursos humanos,soporte,alto,medio
count,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000,11991.00000,11991.000000,11991.000000,11991.000000,11991.000000,11991.000000
mean,6.296581,7.166825,3.802852,200.473522,3.364857,0.154282,0.166041,0.016929,0.057877,0.036361,0.051789,0.057210,0.081394,0.18714,0.056125,0.050121,0.151864,0.082562,0.438746
std,2.410700,1.683426,1.163238,48.727813,1.330240,0.361234,0.372133,0.129012,0.233520,0.187194,0.221610,0.232252,0.273451,0.39004,0.230173,0.218204,0.358904,0.275230,0.496254
min,0.900000,3.600000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.800000,5.700000,3.000000,157.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,6.600000,7.200000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,8.200000,8.600000,5.000000,243.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,1.000000
max,10.000000,10.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000


## Estimar el modelo de Cox e interpretar cada uno de los coeficientes

In [ ]:
cph=CoxPHFitter().fit(datos,'semestre','renunciar')
cph.print_summary()

<lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
      baseline estimation = breslow
   number of observations = 11991
number of events observed = 1991
   partial log-likelihood = -15884.05
         time fit was run = 2023-09-25 15:36:05 UTC

---
                         coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                      
satisfaccion            -0.24       0.78       0.01            -0.26            -0.22                 0.77                 0.80
evaluacion              -0.01       0.99       0.01            -0.04             0.02                 0.96                 1.02
proyectos               -0.29       0.75       0.02            -0.33            -0.25                 0.72                 0.78
promedio_horas_mes       0.00       1.00       0.00             0.00             0.00                 1.00                 1.00
accidente               -1.25       0.29       0.10            -1.44            -1.05                 0.24                 0.35
ascenso_5years          -1.38       0.25       0.36            -2.08            -0.68                 0.13                 0.51
Investigación           -0.35       0.70       0.12            -0.58            -0.12                 0.56                 0.89
administración          -0.13       0.88       0.15            -0.42             0.16                 0.66                 1.17
contabilidad            -0.11       0.90       0.10            -0.32             0.10                 0.73                 1.10
desarrollo de producto   0.01       1.01       0.10            -0.20             0.21                 0.82                 1.24
informática             -0.06       0.94       0.09            -0.24             0.12                 0.79                 1.13
mantenimiento            0.08       1.08       0.07            -0.05             0.21                 0.95                 1.24
mercadotecnia           -0.01       0.99       0.10            -0.22             0.19                 0.81                 1.21
recursos humanos         0.08       1.09       0.10            -0.12             0.29                 0.89                 1.33
soporte                  0.10       1.11       0.07            -0.04             0.24                 0.96                 1.27
alto                    -1.54       0.21       0.15            -1.83            -1.25                 0.16                 0.29
medio                   -0.39       0.68       0.05            -0.48            -0.30                 0.62                 0.74

                         cmp to      z      p   -log2(p)
covariate                                               
satisfaccion               0.00 -26.89 <0.005     526.58
evaluacion                 0.00  -0.76   0.44       1.17
proyectos                  0.00 -14.75 <0.005     161.15
promedio_horas_mes         0.00   3.88 <0.005      13.25
accidente                  0.00 -12.41 <0.005     115.04
ascenso_5years             0.00  -3.88 <0.005      13.21
Investigación              0.00  -3.01 <0.005       8.56
administración             0.00  -0.89   0.37       1.42
contabilidad               0.00  -1.05   0.29       1.77
desarrollo de producto     0.00   0.09   0.93       0.11
informática                0.00  -0.65   0.52       0.95
mantenimiento              0.00   1.23   0.22       2.18
mercadotecnia              0.00  -0.11   0.91       0.14
recursos humanos           0.00   0.81   0.42       1.26
soporte                    0.00   1.42   0.16       2.67
alto                       0.00 -10.36 <0.005      81.15
medio                      0.00  -8.39 <0.005      54.18
---
Concordance = 0.85
Partial AIC = 31802.11
log-likelihood ratio test = 1360.94 on 17 df
-log2(p) of ll-rati

In [ ]:
cph.params_

covariate
satisfaccion             -0.242337
evaluacion               -0.010687
proyectos                -0.291606
promedio_horas_mes        0.001839
accidente                -1.246686
ascenso_5years           -1.380084
Investigación            -0.350547
administración           -0.130262
contabilidad             -0.110158
desarrollo de producto    0.009468
informática              -0.058469
mantenimiento             0.081424
mercadotecnia            -0.011698
recursos humanos          0.083791
soporte                   0.100538
alto                     -1.538901
medio                    -0.391289
Name: coef, dtype: float64

In [ ]:
cph.hazard_ratios_

covariate
satisfaccion              0.784792
evaluacion                0.989370
proyectos                 0.747063
promedio_horas_mes        1.001841
accidente                 0.287456
ascenso_5years            0.251557
Investigación             0.704303
administración            0.877865
contabilidad              0.895692
desarrollo de producto    1.009513
informática               0.943208
mantenimiento             1.084830
mercadotecnia             0.988370
recursos humanos          1.087401
soporte                   1.105766
alto                      0.214617
medio                     0.676185
Name: exp(coef), dtype: float64

### Interpretación de los coeficientes

1. Satisfacción <br>
El p-value es menor a alfa, por lo que rechazamos H0 y la satisfacción  influye en el riesgo de que un empleado renuncie. El riesgo de que un empleado renuncie disminuye en un 22% por cada nivel de satisfacción.
2. Evaluación <br>
El p-value es mayor a alfa, por lo que no rechazamos H0 y la evaluación (que tan bien hace su trabajo) no influye en el riesgo de que un empleado renuncie.
3. Proyectos <br>
El p-value es menor a afa, por lo que rechazamos H0 y el número de proyectos asignados a los empleados influye en el riesgo de que un empleado renuncie. El riesgo de que un empleado renuncie disminuye en un 25% por cada proyecto asignado a un empleado.
4. Promedio Hora Mes <br>
El p-value es menor a alfa, por lo que rechazamos H0 y el número promedio de horas trabajadas al mes influye en el riesgo de que un empleado renuncie. El riesgo de que un empleado renuncie aumenta en un 0.1% por cada hora extra en promedio de trabajo mensual.
5. Accidente <br>
El p-value es menor a alfa, por lo que rechazamos H0 y si un empleado a tenido un accidente dentro de la empresa influye en el riesgo de que un empleado renuncie. Si un empleado ha tenido un accidente, el riesgo de que un empleado renuncie disminuye en un 71% con respecto a los que no han tenido accidentes dentro de la empresa.
6. Ascenso en 5 años <br>
El p-value es menor a alfa, por lo que rechazamos H0 y si un empleado ha sido ascendido durante los últimos 5 años influye en el riesgo de que un empleado renuncie. Si un empleado ha sido ascendido en los últimos 5 años, el riesgo disminuye en un 75% con respecto a los que no han sido ascendidos en los últimos 5 años.
7. Departamento de Investigación <br>
El p-value es menor a alfa, por lo que formar parte del departamento de Investigación influye en el riesgo de que un empleado renuncie. Si un empleado forma parte del departamento de investigación, el riesgo de que un empleado renuncie disminuye en un 30% con respecto al departamento de ventas.
8. Departamento de Adminitración <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Administración no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
9. Departamento de Contabilidad <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Administración no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
10. Departamento de Desarrollo de Producto <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Desarrollo de Producto no influye en el riesgo de que un empleado renuncie. con respecto al departamento de ventas
11. Departamento de Informática <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Informática no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
12. Departamento de Mantenimiento <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Mantenimiento no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
13. Departamento de Mercadotecnia <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Mercadotecnia no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
14. Departamento de Recursos Humanos <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Recursos Humanos no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
15. Departamento de Soporte <br>
El p-value es mayor a alfa, por lo que formar parte del departamento de Soporte no influye en el riesgo de que un empleado renuncie con respecto al departamento de ventas.
16. Salario Alto <br>
El p-value es menor a alfa, por lo que tener un salario alto influye en el riesgo de que un empleado renuncie. Si un empleado tiene salario alto, el riesgo de que un empleado renuncie disminuye en un 79% con respecto a los empleados con salario bajo.
17. Salario Medio <br>
El p-value es menor a alfa, por lo que tener un salario medio influye en el riesgo de que un empleado renuncie. Si un empleado tiene salario medio, el riesgo de que un empleado renuncie disminuye en un 32% con respecto a los empleados con salario bajo.

### ¿Qué recomendación podría hacerle  a  la  empresa  para  que  disminuya  la  cantidad  de  renuncias  de  los empleados?

Recomendaría trabajar con la satisfacción, el ascenso en los últimos 5 años y los salarios. Ya que si bien, no son los únicos que disminuyen el riesgo, considero que son aquellas que más control podrían tener sobre los empleados o las condiciones laborales. Aumentar la satisfacción, la tasa de ascensos y los salarios disminuirá significativamente el riesgo de que un empleado renuncie.

In [ ]:
# Pruebe si el modelo de Cox cumple el supuesto de riesgos proporcionales, de no hacerlo estime el modelo AFT adecuado
cph.check_assumptions(datos, p_value_threshold=0.05)

The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 11991 total observations, 10000 right-censored observations>
         test_name = proportional_hazard_test

---
                             test_statistic      p  -log2(p)
Investigación          km              1.04   0.31      1.70
                       rank            4.36   0.04      4.76
accidente              km              0.03   0.87      0.21
                       rank            1.14   0.29      1.81
administración         km              3.66   0.06      4.17
                       rank            1.53   0.22      2.21
alto                   km              0.37   0.54      0.88
                       rank            0.00   0.96      0.06
ascenso_5years         km              3.66   0.06      4.17
                       rank            6.71   0.01      6.71
contabilidad           km              0.52   0.47      1.09
                       rank            0.71   0.40      1.33
desarrollo de producto km              0.07   0.79      0.34
                       rank            1.10   0.29      1.77
evaluacion             km            443.05 <0.005    324.32
                       rank          427.15 <0.005    312.82
informática            km              0.08   0.77      0.37
                       rank            2.59   0.11      3.22
mantenimiento          km              0.31   0.58      0.79
                       rank            0.91   0.34      1.56
medio                  km              0.44   0.51      0.98
                       rank            0.75   0.39      1.37
mercadotecnia          km              1.24   0.27      1.91
                       rank            1.31   0.25      1.99
promedio_horas_mes     km            270.84 <0.005    199.74
                       rank          307.41 <0.005    226.21
proyectos              km            574.13 <0.005    419.05
                       rank          684.93 <0.005    499.11
recursos humanos       km              0.00   0.99      0.01
                       rank            0.00   1.00      0.00
satisfaccion           km           1257.74 <0.005    912.74
                       rank          845.24 <0.005    614.90
soporte                km              2.68   0.10      3.30
                       rank            3.25   0.07      3.81



1. Variable 'satisfaccion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'satisfaccion' might be incorrect. That is, there
may be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'satisfaccion' using pd.cut, and then specify it in
`strata=['satisfaccion', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'evaluacion' failed the non-proportional test: p-value is <5e-05.

   Advice 1: the functional form of the variable 'evaluacion' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to 

[]

No todas las variables cumplen con el supuesto de riesgo proporcional, por lo que se empleara el modelo AFT.

In [ ]:
from lifelines import ExponentialFitter, WeibullFitter, LogNormalFitter
M_Exp=ExponentialFitter().fit(datos['semestre'],datos['renunciar'])
M_Wei=WeibullFitter().fit(datos['semestre'],datos['renunciar'])
M_LogN=LogNormalFitter().fit(datos['semestre'],datos['renunciar'])

In [ ]:
M_Exp.AIC_,M_Wei.AIC_,M_LogN.AIC_

(15965.45892201228, 12511.512867661068, 11587.92169210356)

In [ ]:
# El menor AIC es del modelo Log Normal, por lo que emplearemos un modelo Log Normal AFT
from lifelines import LogNormalAFTFitter
logAFT=LogNormalAFTFitter().fit(datos,'semestre','renunciar')
logAFT.print_summary()


<lifelines.LogNormalAFTFitter: fitted with 11991 total observations, 10000 right-censored observations>
             duration col = 'semestre'
                event col = 'renunciar'
   number of observations = 11991
number of events observed = 1991
           log-likelihood = -4876.57
         time fit was run = 2023-09-25 16:34:33 UTC

---
                                coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param  covariate                                                                                                                      
mu_    Investigación            0.06       1.06       0.03             0.00             0.11                 1.00                 1.12
       accidente                0.26       1.30       0.02             0.22             0.30                 1.25                 1.35
       administración           0.11       1.11       0.03             0.04             0.17                 1.04                 1.18
       alto                     0.37       1.45       0.03             0.31             0.43                 1.37                 1.53
       ascenso_5years           0.37       1.44       0.06             0.24             0.49                 1.27                 1.63
       contabilidad             0.01       1.01       0.03            -0.04             0.06                 0.96                 1.07
       desarrollo de producto   0.01       1.01       0.02            -0.04             0.05                 0.96                 1.06
       evaluacion               0.01       1.01       0.00            -0.00             0.01                 1.00                 1.01
       informática              0.01       1.01       0.02            -0.03             0.05                 0.97                 1.05
       mantenimiento           -0.02       0.98       0.02            -0.05             0.01                 0.95                 1.01
       medio                    0.08       1.09       0.01             0.06             0.11                 1.06                 1.11
       mercadotecnia            0.02       1.02       0.03            -0.03             0.07                 0.97                 1.07
       promedio_horas_mes      -0.00       1.00       0.00            -0.00            -0.00                 1.00                 1.00
       proyectos                0.10       1.10       0.01             0.09             0.11                 1.09                 1.11
       recursos humanos        -0.03       0.97       0.03            -0.08             0.02                 0.92                 1.02
       satisfaccion             0.07       1.07       0.00             0.06             0.07                 1.06                 1.07
       soporte                 -0.03       0.97       0.02            -0.06             0.01                 0.94                 1.01
       Intercept                0.91       2.50       0.03             0.86             0.97                 2.36                 2.63
sigma_ Intercept               -1.11       0.33       0.02            -1.14            -1.08                 0.32                 0.34

                                cmp to      z      p   -log2(p)
param  covariate                                               
mu_    Investigación              0.00   2.13   0.03       4.92
       accidente                  0.00  12.98 <0.005     125.59
       administración             0.00   3.22 <0.005       9.60
       alto                       0.00  12.49 <0.005     116.53
       ascenso_5years             0.00   5.80 <0.005      27.19
       contabilidad               0.00   0.55   0.58       0.79
       desarrollo de producto     0.00   0.22   0.82       0.28
       evaluacion                 0.00   1.46   0.14       2.80
       informática                0.00   0.51   0.61       0.71
       mantenimiento              0.00  -1.25   0.21       2.25
       medio                      0.00   7.50 <0.005

### Interpretación de los coeficientes AFT

1. Satisfaccion <br>
El p-value es menor a alfa, por lo que rechazamos H0. El nivel de satisfacción aumenta la mediana del tiempo de renuncia en un 7% por cada nivel de satisfacción.
2. Evaluacion <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. El grado de evaluación no influye en la mediana del tiempo de renuncia.
3. Proyectos <br>
El p-value es menor a alfa, por lo que rechazamos H0. El número de proyectos asisgnados a los empleados aumenta la mediana de tiempo de renuncia en un 10% por cada proyecto asignado.
4. Promedio horas mes <br>
El p-value es menor a alfa, por lo que rechazamos H0. El número promedio de horas al mes trabajadas por los empleados disminuye la mediana de tiempo de renuncia en un en un valor cercano a 0%
5. Accidente <br>
El p-value es menor a alfa, por lo que rechazamos H0. Si un trabajador ha sufrido un accidente, la mediana de tiempo de renuncia aumenta en un 30% con respecto a los que no han sufrido un accidente.
6. Ascenso en 5 años <br>
El p-value es menor a alfa, por lo que rechazamos H0. Si un trabajador recibe un ascenso en 5 años, la mediana de tiempo de renuncia aumenta en un 44% con respecto a los que no han tenido ascenso en los últimos 5 años.
7. Departamento de Investigación <br>
El p-value es menor a alfa, por lo que rechazamos H0. Si un trabajador está en el departamento de investigación, la mediana de tiempo de renuncia aumenta en un 6% con respecto al departamento de ventas.
8. Departamento de Administración <br>
El p-value es menor a alfa, por lo que rechazamos H0. Si un trabajador está en el departamento de investigación, la mediana de tiempo de renuncia aumenta en un 11% con respecto al departamento de ventas.
9. Departamento de Contabilidad <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de contabilidad no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
10. Departamento de Desarrollo de producto <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de desarrollo de producto no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
11. Departamento de Informática <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de informática no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
12. Departamento de Mantenimiento <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de mantenimiento no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
13. Departamento de Mercadotecnia <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de mercadotecnia no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
14. Departamento de Recursos Humanos <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de recursos humanos no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
15. Departamento de Soporte <br>
El p-value es mayor a alfa, por lo que no rechazamos H0. Formar parte del departamento de soporte no influye en la mediana del tiempo de renuncia con respecto al departamento de ventas.
16. Salario alto <br>
El p-value es menor a alfa, por lo que rechazamos H0. Un empleado con salario alto aumenta la mediana del tiempo de renuncia en un 45% con respecto a los empleados con salario bajo.
17. Salario medio <br>
El p-value es mayor a alfa, por lo que rechazamos H0. Un empleado con salario medio aumenta la mediana del tiempo un 9% con respecto a los empleados con salario bajo.